In [1]:
import os
import numpy
from pyscf import gto
from pyscf import scf
from pyscf import dft


mol = gto.Mole()
mol.verbose = 3
mol.atom = [
 ["C",  ( 0.000000,  0.418626, 0.000000)],
 ["H",  (-0.460595,  1.426053, 0.000000)],
 ["O",  ( 1.196516,  0.242075, 0.000000)],
 ["N",  (-0.936579, -0.568753, 0.000000)],
 ["H",  (-0.634414, -1.530889, 0.000000)],
 ["H",  (-1.921071, -0.362247, 0.000000)]
]
mol.basis = {"H": '6-311++g**',
             "O": '6-311++g**',
             "N": '6-311++g**',
             "C": '6-311++g**',
             }
mol.build()

# First state calculation with DFT
a = dft.UKS(mol)
a.xc='b3lyp'
# Store molecular orbital information into chkfile
a.chkfile='nh2cho_s0.chkfile'
a.scf()
mo0 = a.mo_coeff
occ0 = a.mo_occ

# Set initial ouoccupation pattern for excited state
occ0[0][11] = 0.0
occ0[0][12] = 1.0

# Second state calculation with DFT 
b = dft.UKS(mol)
b.xc='b3lyp'
# Store molecular orbital information into another chkfile
b.chkfile='nh2cho_s1.chkfile'
dm = b.make_rdm1(mo0, occ0)
# Use mom method to determine occupation number
scf.addons.mom_occ_(b, mo0, occ0)
b.scf(dm)

# Read the MO coefficients and occupation numbers from chkfile.
# So the calculation of electronic coupling can be carried out 
# standalone use chkfiles.
mo0 = scf.chkfile.load('nh2cho_s0.chkfile', 'scf/mo_coeff')
occ0 = scf.chkfile.load('nh2cho_s0.chkfile', 'scf/mo_occ')
mo1 = scf.chkfile.load('nh2cho_s1.chkfile', 'scf/mo_coeff')
occ1 = scf.chkfile.load('nh2cho_s1.chkfile', 'scf/mo_occ')

mf = scf.UHF(mol)
# Calculate overlap between two determiant <I|F>
s, x = mf.det_ovlp(mo0, mo1, occ0, occ1)

converged SCF energy = -169.865834514445  <S^2> = 1.0538059e-10  2S+1 = 1


Overwritten attributes  get_occ  of <class 'pyscf.dft.uks.UKS'>


converged SCF energy = -169.622254719194  <S^2> = 1.0030038  2S+1 = 2.2387531


In [2]:
s

0.022180120282568012

In [3]:
x

(array([[ 1.00000004e+00, -2.64303707e-07,  6.58391749e-07,
          3.48977845e-04, -1.15160761e-04, -1.44517898e-04,
         -5.80952808e-05,  1.74927034e-04, -6.61186859e-17,
          4.84368453e-05,  2.42032122e-17,  9.83653130e-04],
        [ 3.73229221e-07,  1.00000023e+00, -2.25319766e-06,
         -1.54914761e-05,  2.25757876e-05, -1.18250840e-04,
         -7.65298227e-05, -9.95684018e-05, -9.49268371e-17,
         -3.27612360e-05, -1.70526516e-16,  2.09990356e-03],
        [-5.61256338e-07,  2.30839733e-06,  1.00000016e+00,
         -2.44541039e-04,  1.04840887e-04,  1.28417670e-05,
         -6.83460249e-05,  2.29633439e-04,  5.41900175e-17,
          1.16991477e-04, -1.36977252e-16,  1.56323983e-04],
        [-3.36118812e-04,  8.76785398e-06,  2.22306225e-04,
          9.90917926e-01,  1.31672405e-01,  2.63920854e-04,
         -5.29098559e-03,  1.14241481e-02, -4.43766738e-15,
          1.09758278e-02,  4.92267270e-16,  7.72432826e-02],
        [ 1.51437938e-04, -4.5090166